In [1]:
# -*- coding:utf-8 -*-
import pymongo
import requests
import locachoice_utils

def getlocation(addr, city='广州'):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    try:
        params = {'output': 'json', 'ak': '0ZqFawNu58dvih0zpLvBNKDdSLGX6wGC', 'address': addr, 'city': city }
        r = requests.get(url, params)
        print(r.url)
        if r.status_code == 200 or r.status_code == 304:
            print(r.json()['result']['location'])
            return r.json()['result']['location']
        else:
            print(r.status_code)
            return r.status_code
    except Exception as e:
        print("ERROR:", e)
        return 404

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_mongo_crawl_coll():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn['crawl_58_tianhe_shangpu']

def get_mongo_etl_coll():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn['etl_58_tianhe_shangpu']

def clean_etl_coll():
    db = get_monggo_db()
    re = db.drop_collection("etl_58_tianhe_shangpu")
    print(re)

def get_mongo_result_coll():
    conn = get_monggo_db()
    return conn['result_tianhe_area_shangpu_price']

def get_mongo_result_grids():
    conn = get_monggo_db()
    return conn['result_grids']    
            
def get_area_from_desc(desc):
    start = desc.find('区域：')
    if start == -1:
        print('has not area field.')
        return ''
    end = desc.find('：', start+6)
    ret = desc[start:end]
    ret = ret.replace('区域：', '')
    ret = ret.replace('天河\r\n', '').strip()
    end = ret.find('\r\n')
    ret = ret[:end].strip()
#     print(ret)    
    return ret

def get_centiares_from_desc(desc):
    start = desc.find('面积：')
    if start == -1:
        print('has not centiares field.')
        return '0'    
    end = desc.find('㎡', start)
    ret = desc[start:end]
    ret = ret.replace('面积：', '')
    ret = ret.replace('㎡', '').strip()
#     print(ret)    
    return ret

def get_total_price_from_desc(desc):
    start = desc.find('租金：')
    if start == -1:
        print('has not price field.')
        return '0'        
    end = desc.find('\r\n', start)
    end = desc.find('\r\n', end+2)
    ret = desc[start:end]
    ret = ret.replace('租金：', '').strip()
    if '元/㎡/天' in ret:
        print(ret)
        ret = ret.replace('元/㎡/天', '').strip()
        fret = float(ret)
        if fret > 55:
            pass
        else:
            ret += '元/㎡/天'
    elif '元/月' in ret:
        ret = ret.replace('元/月', '').strip()
    else:
        ret = '0'
        print('price type error...')    
#     print(ret)    
    return ret

def get_type_from_desc(desc):
    start = desc.find('类型：')
    if start == -1:
        print('has not type field.')
        return ''    
    end = desc.find('\r\n', start)
    ret = desc[start:end]
    ret = ret.replace('类型：', '').strip()    
#     print(ret)    
    return ret

def get_near_from_desc(desc):
    start = desc.find('临近：')
    if start == -1:
        print('has not type field.')
        return ''    
    end = desc.find('\r\n', start)
    ret = desc[start:end]
    ret = ret.replace('临近：', '').strip()    
#     print(ret)    
    return ret

def etl_tianhe_sp():
    src_coll = get_mongo_crawl_coll()
    dst_coll = get_mongo_etl_coll()
    cursor = src_coll.find()
    for cur in cursor:
        desc = cur.get('description')
        centiares = int(float(get_centiares_from_desc(desc)))  
        strpri = get_total_price_from_desc(desc)
        if '元/㎡/天' in strpri:
            floatpri = float(strpri.replace('元/㎡/天', ''))
            totalpri = int(floatpri * 30 * centiares)
            unitpri = int(floatpri * 30)
        else:
            totalpri = int(float(strpri))
            if centiares != 0:
                unitpri = totalpri // centiares
            else:
                unitpri = 0
    
        doc = {
            'name': cur.get('name'),
            'area': get_area_from_desc(desc),
            'centiares': centiares,
            'total_price': totalpri,
            'unit_price': unitpri,
            'type': get_type_from_desc(desc),
            'near': get_near_from_desc(desc),
        }
        dst_coll.insert_one(doc)

def remove_nvalid_records():
    etl_coll = get_mongo_etl_coll()
    # 删除区域无效数据
    etl_coll.delete_many({'area': '临'})
    etl_coll.delete_many({'area': '地'})
    etl_coll.delete_many({'area': '类'})
    etl_coll.delete_many({'area': '面'})    
    #价格筛选
    etl_coll.delete_many({'unit_price': {'$lte': 18}})
    etl_coll.delete_many({'unit_price': {'$gte': 1500}, 'centiares': {'$lte': 8}})
    
def create_tianhe_area_shangpu_price():
    db = get_monggo_db()
    re = db.drop_collection("result_tianhe_area_shangpu_price")
    print(re)
    dst_coll = db["result_tianhe_area_shangpu_price"]
    src_coll = get_mongo_etl_coll()
    
    areas = src_coll.distinct('area')
    for area in areas:
        cursor = src_coll.find({'area': area})
        i = 0
        total = 0
        for cur in cursor:
            i += 1
            total += cur.get('unit_price')        

        doc = {
            'area': area,
            'price': total // i, 
        }
        
        # 区域名称修正
        area = area.replace('体育中心', '天河体育中心')
        area = area.replace('燕塘', '燕塘广场')
        area = area.replace('沙太南', '沙太南路')    
        area = area.replace('珠吉', '珠吉路')
        
        location = getlocation(area)
        if isinstance(location, int):
            print('can not get location:', area)
            doc['lng'] = 0
            doc['lat'] = 0
        else:
            doc['lng'] = location.get('lng')
            doc['lat'] = location.get('lat')
            
        doc['gridid'] = locachoice_utils.get_grid_id(doc.get('lng'), doc.get('lat'))       
        dst_coll.insert_one(doc)
        
def fill_tianhe_shangpu_fields_in_grids():
    grids_coll = get_mongo_result_grids()
    cursor = grids_coll.find({'shangpu_rent': {'$exists': False}})
    for cur in cursor:
        toset_doc = {
            'shangpu_rent': None,
        }
        print(toset_doc)
        grids_coll.update_one({'gridid': cur.get('gridid')}, {'$set': toset_doc})
  
def export_shangpu_rent2grids():
    coll = get_mongo_result_coll()
    grids_coll = get_mongo_result_grids()
    gridids = coll.distinct('gridid')
    for gridid in gridids:
        if gridid == -1:
            continue
        total_price = 0
        count = 0
        cursor = coll.find({'gridid': gridid})
        for cur in cursor:
            count += 1
            total_price += cur.get('price')
        shangpu_rent = total_price // count
        grids_coll.update_one({'gridid': gridid}, {'$set': {'shangpu_rent': shangpu_rent}})
    fill_tianhe_shangpu_fields_in_grids()      
        
if __name__ == '__main__':
#     clean_etl_coll()
#     etl_tianhe_sp()
#     remove_nvalid_records()
#     create_tianhe_area_shangpu_price()
#     export_shangpu_rent2grids()
    print('ended ended')
    


{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_rent': None}
{'shangpu_r